# SF Salaries Exercise - Solutions

Welcome to a quick exercise for you to practice your pandas skills! We will be using the [SF Salaries Dataset](https://www.kaggle.com/kaggle/sf-salaries) from Kaggle! Just follow along and complete the tasks outlined in bold below. The tasks will get harder and harder as you go along.

## Import pandas as pd

In [1]:
import pandas as pd

## Read `Salaries.csv` as a DataFrame called `salaries`

In [2]:
salaries = pd.read_csv("Salaries.csv", low_memory=False)

## Check the head of the DataFrame

In [3]:
salaries.head()

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
0,1,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0.0,400184.25,NaN,567595.43,567595.43,2011,NaN,San Francisco,NaN
1,2,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,NaN,538909.28,538909.28,2011,NaN,San Francisco,NaN
2,3,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.6,NaN,335279.91,335279.91,2011,NaN,San Francisco,NaN
3,4,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916.0,56120.71,198306.9,NaN,332343.61,332343.61,2011,NaN,San Francisco,NaN
4,5,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.6,9737.0,182234.59,NaN,326373.19,326373.19,2011,NaN,San Francisco,NaN


## Use the `.info()` method to find out how many entries there are

In [4]:
salaries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148654 entries, 0 to 148653
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Id                148654 non-null  int64  
 1   EmployeeName      148654 non-null  object 
 2   JobTitle          148654 non-null  object 
 3   BasePay           148049 non-null  object 
 4   OvertimePay       148654 non-null  object 
 5   OtherPay          148654 non-null  object 
 6   Benefits          112495 non-null  object 
 7   TotalPay          148654 non-null  float64
 8   TotalPayBenefits  148654 non-null  float64
 9   Year              148654 non-null  int64  
 10  Notes             0 non-null       float64
 11  Agency            148654 non-null  object 
 12  Status            38119 non-null   object 
dtypes: float64(3), int64(2), object(8)
memory usage: 14.7+ MB


<span style="color:red">**It appears that some columns need further preprocessing:**</span>
> - `BasePay`, `OvertimePay`, `OtherPay`, `Benefits`: need to be **cleaned** and **converted** to numerical dtype,
> -  `Notes`, `agency`, `Status`: need to be **dropped**.

## Data cleaning and preprocessing

In [5]:
# Drop columns that are irrelevant to analysis:
cols_to_drop = ["Notes", "Agency", "Status"]
salaries.drop(columns=cols_to_drop, inplace=True)

In [6]:
# The "BasePay" and "Benefits" columns have some missind data:
salaries.isna().sum()

Id                      0
EmployeeName            0
JobTitle                0
BasePay               605
OvertimePay             0
OtherPay                0
Benefits            36159
TotalPay                0
TotalPayBenefits        0
Year                    0
dtype: int64

In [7]:
# Replace all missing data in "BasePay" and  "Benefits" with zero:
salaries.loc[salaries["BasePay"].isna(), ["BasePay"]] = 0
salaries.loc[salaries["Benefits"].isna(), ["Benefits"]] = 0

In [8]:
# Recheck the data:
salaries.isna().sum()

Id                  0
EmployeeName        0
JobTitle            0
BasePay             0
OvertimePay         0
OtherPay            0
Benefits            0
TotalPay            0
TotalPayBenefits    0
Year                0
dtype: int64

In [9]:
# Rows to be cleaned:
rows_to_clean = ["BasePay", "OvertimePay", "OtherPay", "Benefits"]

In [10]:
# Define a function to detect and convert numerical data:
def cast_into_float(x):
    "Casts data into float, otherwise returns a 'None' as a str"
    try:
        return float(x)
    except ValueError:
        return "missing!"

In [11]:
# Convert invalid data into float or "None":
for row in rows_to_clean:
    salaries[row] = salaries[row].apply(cast_into_float)

In [12]:
# Check for the dtypes in each column after conversion:
for row in rows_to_clean:
    invalid_rows = salaries[row].apply(lambda x: type(x)).value_counts()
    print(f"{row} has the following dtypes:\n{invalid_rows}\n")

BasePay has the following dtypes:
<class 'float'>    148650
<class 'str'>           4
Name: BasePay, dtype: int64

OvertimePay has the following dtypes:
<class 'float'>    148650
<class 'str'>           4
Name: OvertimePay, dtype: int64

OtherPay has the following dtypes:
<class 'float'>    148650
<class 'str'>           4
Name: OtherPay, dtype: int64

Benefits has the following dtypes:
<class 'float'>    148650
<class 'str'>           4
Name: Benefits, dtype: int64



<span style="color:red">**The rows that contain invalid numerical data are the same IN THE 4 COLUMNS, they need to be dropped!**</span>

In [13]:
# Let's make sure of these findings:
salaries.loc[salaries["Benefits"] == "missing!", rows_to_clean]

,BasePay,OvertimePay,OtherPay,Benefits
148646,missing!,missing!,missing!,missing!
148650,missing!,missing!,missing!,missing!
148651,missing!,missing!,missing!,missing!
148652,missing!,missing!,missing!,missing!


In [14]:
# Drop rows with invalid numerical data:
rows_to_drop = salaries.query("BasePay == 'missing!'").index
salaries.drop(index=rows_to_drop, inplace=True)

In [15]:
# FINALLY, set the correct data type:
for row in rows_to_clean:
    salaries[row] = pd.to_numeric(salaries[row])

## `.info()` REVISITED!

In [16]:
salaries.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148650 entries, 0 to 148653
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Id                148650 non-null  int64  
 1   EmployeeName      148650 non-null  object 
 2   JobTitle          148650 non-null  object 
 3   BasePay           148650 non-null  float64
 4   OvertimePay       148650 non-null  float64
 5   OtherPay          148650 non-null  float64
 6   Benefits          148650 non-null  float64
 7   TotalPay          148650 non-null  float64
 8   TotalPayBenefits  148650 non-null  float64
 9   Year              148650 non-null  int64  
dtypes: float64(6), int64(2), object(2)
memory usage: 12.5+ MB


<span style="color:red">**All data are in the right format, WE ARE READY TO GO!**</span>

## Use the `.describe()` method to get some statistics for the data

In [17]:
salaries.drop(columns=["Id","Year"]).describe().T

,count,mean,std,min,25%,50%,75%,max
BasePay,148650.0,66055.506718,42885.919334,-166.01,32798.5500,64862.310,94609.7150,319275.01
OvertimePay,148650.0,5066.059886,11454.380559,-0.01,0.0000,0.000,4658.1750,245131.88
OtherPay,148650.0,3648.767297,8056.601866,-7058.59,0.0000,811.270,4236.0650,400184.25
Benefits,148650.0,18924.742068,17165.229731,-33.89,0.0000,23217.135,33468.9800,96570.66
TotalPay,148650.0,74770.333901,50516.196009,-618.13,36169.9550,71427.240,105842.9600,567595.43
TotalPayBenefits,148650.0,93695.075969,62792.497391,-618.13,44073.3375,92406.020,132877.9925,567595.43


## What is the average `BasePay`?

<span style="color:red">**There are 2 approaches:**</span>

In [18]:
# Calculate the average BasePay directly:
round(salaries["BasePay"].mean(), 2)

66055.51

In [19]:
# Exclude rows with zero BasePay and calculate the average:
is_base_pay_zero = salaries["BasePay"] != 0
round(salaries.loc[is_base_pay_zero, ["BasePay"]].mean(), 2)

BasePay    66912.11
dtype: float64

## What is the highest amount of `OvertimePay` in the dataset?

In [20]:
salaries['OvertimePay'].max()

245131.88

## What is the job title of  `JOSEPH DRISCOLL`? Note: Use all caps, otherwise you may get an answer that doesn't match up (there is also a lowercase Joseph Driscoll)

In [21]:
salaries.query("EmployeeName == 'JOSEPH DRISCOLL'")["JobTitle"]

24    CAPTAIN, FIRE SUPPRESSION
Name: JobTitle, dtype: object

## How much does JOSEPH DRISCOLL make (including benefits)?

In [22]:
salaries.query("EmployeeName == 'JOSEPH DRISCOLL'")["TotalPayBenefits"]

24    270324.91
Name: TotalPayBenefits, dtype: float64

## What is the name of highest paid person (including benefits)?

In [23]:
# Get the highest value in "TotalPayBenefits":
highest_total_pay_with_benefits = salaries["TotalPayBenefits"].max()
highest_total_pay_with_benefits

567595.43

In [24]:
# Get the person that is paid this amount:
salaries.query("TotalPayBenefits == @highest_total_pay_with_benefits")["EmployeeName"]

0    NATHANIEL FORD
Name: EmployeeName, dtype: object

## What is the name of lowest paid person (including benefits)? Do you notice something strange about how much he or she is paid?

In [25]:
# Get the lowest value in "TotalPayBenefits":
lowest_total_pay_with_benefits = salaries["TotalPayBenefits"].min()
lowest_total_pay_with_benefits

-618.13

In [26]:
# Get the person that is paid this amount:
salaries.query("TotalPayBenefits == @lowest_total_pay_with_benefits")["EmployeeName"]

148653    Joe Lopez
Name: EmployeeName, dtype: object

<span style="color:red">**This guy is literally LOSING MONEY TO THE COMPANY!**</span>

## What was the average (mean) `BasePay` of all employees per year? (2011-2014)?

<span style="color:red">**There are 2 approaches:**</span>

In [27]:
# Calculate the whole avaerage:
round(salaries.query("2011<= Year <= 2014")["BasePay"].mean(), 2)

66055.51

In [28]:
# Group the data by year and calculate each year's average:
round(salaries[["BasePay", "Year"]].groupby(by="Year").mean(), 2)

,BasePay
Year,
2011,63595.96
2012,65436.41
2013,68509.83
2014,66564.42


## How many unique job titles are there?

In [29]:
salaries["JobTitle"].nunique()

2158

## What are the top 5 most common jobs?

In [30]:
salaries["JobTitle"].value_counts()[:5]

Transit Operator                7036
Special Nurse                   4389
Registered Nurse                3736
Public Svc Aide-Public Works    2518
Police Officer 3                2421
Name: JobTitle, dtype: int64

## How many Job Titles were represented by only one person in 2013? (e.g. Job Titles with only one occurence in 2013?)

In [31]:
# Slice the data according to the given year:
job_titles_in_2013 = salaries.query("Year == 2013")[["EmployeeName", "JobTitle"]]
job_titles_in_2013.head()

,EmployeeName,JobTitle
72925,Gregory P Suhr,Chief of Police
72926,Joanne M Hayes-White,"Chief, Fire Department"
72927,Samson Lai,"Battalion Chief, Fire Suppress"
72928,Ellen G Moffatt,Asst Med Examiner
72929,Robert L Shaw,"Dep Dir for Investments, Ret"


In [32]:
# Group the data by JobTitle and count the occurrence of each title:
job_title_counts_in_2013 = job_titles_in_2013.groupby(by="JobTitle").count()
job_title_counts_in_2013.head()

,EmployeeName
JobTitle,
ASR Senior Office Specialist,21
ASR-Office Assistant,15
Account Clerk,90
Accountant I,3
Accountant II,77


In [33]:
# Count how many title represented by only one person:
(job_title_counts_in_2013 == 1).sum()

EmployeeName    202
dtype: int64

## How many people have the word Chief in their job title? (This is pretty tricky)

In [34]:
word_to_search = "Chief"

In [35]:
is_chief = salaries["JobTitle"].apply(lambda x: True if word_to_search.casefold() in x.casefold() else False)
is_chief.sum()

627

## Bonus: Is there a correlation between length of the Job Title string and Salary?

In [36]:
salaries["JobTitleLength"] = salaries["JobTitle"].apply(lambda x: len(x))
salaries["JobTitleLength"].head()

0    46
1    31
2    31
3    36
4    44
Name: JobTitleLength, dtype: int64

In [37]:
salaries[["JobTitleLength", "TotalPayBenefits"]].corr()

,JobTitleLength,TotalPayBenefits
JobTitleLength,1.00000,-0.03692
TotalPayBenefits,-0.03692,1.00000


<span style="color:red">**There is NO CORRELATION between `JobTitleLength` and `TotalPayBenefits`!**</span>

# Great Job!